In [ ]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.premierleague.com/communities'
r = requests.get(url)
html = r.text
soup = BeautifulSoup(html,'html5lib')
ul_tag = soup.find_all('span',attrs={'class':"title"})

title = [i.text for i in ul_tag]

print(title)

In [2]:
import numpy as np
import pandas as pd
import re
import time
from datasketch import MinHash, MinHashLSHForest